<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Testing injection and recovery of "supernovaish" lightcurves</b> <br>
Contact author: Michael Wood-Vasey <br>
Last verified to run: 2023-05-24 <br>
LSST Science Pipelines version: Weekly 2023_20 + source_injection tickets/DM-34253 <br>
Container Size: large <br>
Targeted learning level: intermediate <br>

If we inject supernova lightcurves into a series of images and then run subtractions, what do we get back?

The details of a supernova aren't key here, so we'll just use some Gaussian model of a lightcurve that has a rise and fall timescale of a few months.

Note: This Notebook is written below the PipelineTask level.  Rather is uses individual Tasks directly and reads/writes output products to the butler.  This is pedagogically useful to understand how that works, and pratically helpful in working with the evolving `source_injection` package.  However, this structure is not scalable to larger runs (100+ images).  Such large-scale runs should be done as part of an integrated Task that can be connected and run through the large-scale cluster jobs submission.

1. [ ] Find set of images
2. [ ] Simulate N supernovae over the T years of survey
3. [ ] Simulate with peak magnitdues from 24 to 18
4. [ ] Run subtractions
5. [ ] Run measurement
6. [ ] Match sources
7. [ ] Compare injection catalog to recovered values

In [ ]:
import os

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
import astropy.units as u

import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure
from lsst.afw.math import Warper, WarperConfig
import lsst.geom as geom
from lsst.daf.butler import Butler, DimensionUniverse, DatasetType, CollectionType
from lsst.daf.butler.registry import MissingCollectionError
from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask
from lsst.pipe.tasks.makeWarp import MakeWarpConfig, MakeWarpTask
import lsst.sphgeom

Here are the parts that we're going to use from the `source_injection` package

In [ ]:
from lsst.source.injection.inject_engine import generate_galsim_objects, inject_galsim_objects_into_exposure

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')
%matplotlib inline

In [ ]:
INJECTED_IMAGES_EXIST = False
SUBTRACTIONS_OF_INJECTED_IMAGE_EXIST = False

We can run this on either DC2 or HSC by choosing appropriate RA, Dec
Currently (2023-05-26) DC2 is only available at the IDF and HSC is only available at the USDF, so we split by site.

In [ ]:
SITE = "USDF"

survey_site = {"USDF": "HSC", "IDF": "DC2"}
repo_site = {"USDF": "/repo/main", "IDF": "dp02"}
collection_site = {"USDF": "HSC/runs/RC2/w_2023_15/DM-38691", "IDF": "2.2i/runs/DP0.2"}

ra_dec_survey = {"HSC": (150, 2.5), "DC2": (55, -30)}

In [ ]:
collection = collection_site[SITE]
repo_config = repo_site[SITE]

user = os.getenv("USER")
output_collection = f"u/{user}/inject_lightcurves"

In [ ]:
butler = Butler(repo_config, run=output_collection, collections=[output_collection, collection])

In [ ]:
print(butler.registry.getDatasetType("calexp"))

In [ ]:
# Do a spatial query for calexps using HTM levels following example in 04b_Intermediate_Butler_Queries.ipynb
ra, dec = ra_dec_survey[survey_site[SITE]]
band = "i"

In [ ]:
def htm_from_ra_dec_level(ra, dec, level):
    pixelization = lsst.sphgeom.HtmPixelization(level)
    htm_id = pixelization.index(
        lsst.sphgeom.UnitVector3d(
            lsst.sphgeom.LonLat.fromDegrees(ra, dec)
        )
    )
    return htm_id

In [ ]:
level = 20  # the resolution of the HTM grid
htm_id = htm_from_ra_dec_level(ra, dec, level)

In [ ]:
# HTM7 will be our spatial dimension for catalog sharding
htm7 = htm_from_ra_dec_level(ra, dec, level=7)

In [ ]:
print(htm7, htm_id)

In [ ]:
parent_level = htm_id // 10
htm_ids = [parent_level * 10 + i for i in [0, 1, 2, 3]]

In [ ]:
htm_ids

In [ ]:
hi = htm_ids[0]

# dataset_refs is an iterator, but each query is only a few hundred results,
#   so convert to a list for future convenience
htm_kwargs = {}
htm_kwargs[f"htm{level}"] = hi
dataset_refs = list(butler.registry.queryDatasets("calexp", dataId={"band": band}, **htm_kwargs))
dataset_refs = set(dataset_refs)

In [ ]:
ra, dec

In [ ]:
for hi in htm_ids[1:]:
    htm_kwargs = {}
    htm_kwargs[f"htm{level}"] = hi
    dr = list(butler.registry.queryDatasets("calexp", dataId={"band": band}, **htm_kwargs))
    dataset_refs = dataset_refs.intersection(set(dr))

In [ ]:
dataset_refs = list(dataset_refs)
# Sort by visitId to get a loose time order
ids_visit = [dr.dataId["visit"] for dr in dataset_refs]
dataset_refs = [dataset_refs[idx] for idx in np.argsort(ids_visit)]

print(dataset_refs)

In [ ]:
print(f"Found {len(list(dataset_refs))} calexps")

In [ ]:
visit_table = butler.get("visitTable") #, dataset_refs[0].dataId)

We should find 17 calexps for DC2.  (RA, Dec) = (55, -30)

We should find 44 calexps for HSC COSMOS.  (RA, Dec) = (150, +2.5)

## Inject object

In [ ]:
# Simulate star
# Start with a constant magnitude
BASE_MAG_STAR = 21  # u.mag
BASE_STAR_RA, BASE_STAR_DEC = ra + 0.01, dec + 0.01
if survey_site[SITE] == "DC2":
    BASE_MJD = 60_000
elif survey_site[SITE] == "HSC":
    BASE_MJD = 56740

# Wrap with a function to allow accessing consistent with a future expansion to time.
def mag_star(phase=0 * u.d, decay_rate=0.1 * u.mag / u.d, peak_mag=BASE_MAG_STAR):
    """
    Simple linear decay function
    
    All negative phase points are set to 0 flux or np.inf mag.
    """
    relative_lightcurve = phase * decay_rate
    # We do this asa condition array instead of using
    # a (phase < 0) * np.inf above because that wouldn't work because np.inf * 0 = np.nan
    relative_lightcurve[phase < 0] = np.inf

    return peak_mag * u.mag + relative_lightcurve


def ra_star(phase=0 * u.d, pm_ra=0.1 * u.arcsec / u.yr, base_ra = BASE_STAR_RA):
    return base_ra * u.degree + phase * pm_ra


def dec_star(phase=0 * u.d, pm_dec=0.1 * u.arcsec / u.yr, base_dec = BASE_STAR_DEC):
    return base_dec * u.degree + phase * pm_dec

Create a catalog for each visit

In [ ]:
injection_catalog = {}
base_mjd = BASE_MJD

for dr in dataset_refs:
    visit = dr.dataId["visit"]
    mjd = visit_table.loc[visit]["expMidptMJD"]
    phase = (mjd - base_mjd) * u.d
    this_cat = Table()
    this_cat["ra"] = [ra_star(phase).to_value(u.degree)]
    this_cat["dec"] = [dec_star(phase).to_value(u.degree)]
    this_cat["mag"] = [mag_star(phase).to_value(u.mag)]
    this_cat["source_type"] = ["DeltaFunction"]
    this_cat["index"] = ["test"]
    injection_catalog[visit] = this_cat

### Register a Collection to write injection catalog and injected images to

In [ ]:
if not INJECTED_IMAGES_EXIST:
    try:
        butler.removeRuns([output_collection])
    except MissingCollectionError:
        print("Writing into a new RUN collection")
        pass
    else:
        print("Prior RUN collection located and successfully removed")

_ = butler.registry.registerCollection(output_collection, type=CollectionType.RUN)

## Register a DatasetType for injected catalog

The call to `registerDatasetType` will report "False" if the DatasetType already exists

In [ ]:
# "injection_" is for the input catalog that will be used for _injection_ into the images
# "injected_" will be for the products that have been _injected_
injection_dataset_type =  DatasetType(
    "injection_time_domain_catalog",
    dimensions=["htm7", "band", "visit"],
    storageClass="ArrowAstropy",
    universe=DimensionUniverse(),
)

butler.registry.registerDatasetType(injection_dataset_type)

In [ ]:
instrument = dataset_refs[0].dataId["instrument"]
for visit, cat in injection_catalog.items():
    data_id = {"htm7": htm7, "visit": visit, "band": band, "instrument": instrument}
    butler.put(cat, "injection_time_domain_catalog", dataId=data_id)

In [ ]:
foo = butler.registry.queryDatasets("injection_time_domain_catalog")
print(list(foo))

## Inject fake sources on images

In [ ]:
data_id

In [ ]:
def injected_and_save_image(data_id,
                            butler: Butler,
                            image_dataset_type="calexp",
                            catalog_dataset_type="injection_time_domain_catalog",
                            mask_plane_name: str = "INJECTED",
                            calib_flux_radius: float = 12.0,
                            draw_size_scale: float = 1.0,
                            draw_size_max: int = 1000,
                            verbose=True):
    "Load an image, injected a catalog, and save to a new collection"
    calexp = butler.get(image_dataset_type, dataId=data_id)
    
    # These aren't the RA, Dec I want, because these are the boresight.
    # I want the center of the detector.
    ra = visit_table.loc[dataset_refs[0].dataId["visit"]]["ra"]
    dec = visit_table.loc[dataset_refs[0].dataId["visit"]]["decl"]
    # htm7 = htm_from_ra_dec_level(ra, dec, level=7)
    cat_data_id = {}
    cat_data_id["htm7"] = htm7
    cat_data_id["visit"] = data_id["visit"]
    cat_data_id["instrument"] = data_id["instrument"]
    cat_data_id["band"] = data_id["band"]
    if verbose:
        print(cat_data_id)
    injection_catalog = butler.get(catalog_dataset_type, dataId=cat_data_id)

    injected_object_generator = generate_galsim_objects(
        injection_catalog=injection_catalog,
        wcs=calexp.getWcs(),
        photo_calib=calexp.getPhotoCalib(),
        fits_alignment = "wcs",
    )

    # ( draw_sizes, common_bounds, fft_size_errors, psf_compute_errors, ) = 

    if verbose:
        print(f"Inserting objects into {calexp}")
        
    _ = inject_galsim_objects_into_exposure(
        calexp,
        injected_object_generator,
        mask_plane_name=mask_plane_name,
        calib_flux_radius=calib_flux_radius,
        draw_size_scale=draw_size_scale,
        draw_size_max=draw_size_max,
    )
    
    if verbose:
        print("Saving newly injected image: ", calexp)
        print(butler)

    butler.put(calexp, f"injected_{image_dataset_type}", dataId=data_id)

In [ ]:
print(butler.run)

In [ ]:
butler.registry.queryDatasetTypes("injection*")

In [ ]:
if not INJECTED_IMAGES_EXIST:
    for dr in dataset_refs:
        injected_and_save_image(dr.dataId, butler)

In [ ]:
foo = butler.registry.queryDatasets("injected_calexp")
print(list(foo))

# Run subtraction between calexps 2-N and calexp 1.

In [ ]:
config = AlardLuptonSubtractConfig()
task = AlardLuptonSubtractTask(config=config)

In [ ]:
template = butler.get("calexp", dataset_refs[0].dataId)

In [ ]:
def subtract(science, template, source_catalog):
    warper_config = WarperConfig()
    warper = Warper.fromConfig(warper_config)

    science_wcs = science.getWcs()
    science_bbox = science.getBBox()

    science.visitInfo.id
    
    # Add PSF.  I think doing this directly without warping is wrong.  At least the x,y mapping should be updated
    warped_template = warper.warpExposure(science_wcs, template, destBBox=science_bbox)
    warped_template.setPsf(template.getPsf())
    
    # Now let's do the subtraction
    subtraction = task.run(warped_template, science, source_catalog)
    
    return subtraction


def detect(science, subtraction):
    # Run detection on subtraction
    detect_and_measure_config = DetectAndMeasureConfig()
    detect_and_measure_task = DetectAndMeasureTask(config=detect_and_measure_config)

    detect_and_measure = detect_and_measure_task.run(science, subtraction.matchedTemplate, subtraction.difference)

    return detect_and_measure

In [ ]:
def subtract_and_detect(data_id: dict,
                        template: lsst.afw.image.ExposureF,
                        butler: Butler):
    """
    Subtract template image from image referred to by data_id and run detection.
    
    Butler needs to be writeable to store output of subtraction and detection.
    """
    science = butler.get("injected_calexp", dr.dataId)
    source_catalog = butler.get("src", dataId=dr.dataId)

    subtraction = subtract(science, template, source_catalog)
    butler.put(subtraction.difference, "goodSeeingDiff_differenceExp", dataId=dr.dataId)

    detection_catalog = detect(science, subtraction)
    butler.put(detection_catalog.diaSources, "goodSeeingDiff_diaSrc", dataId=dr.dataId)

In [ ]:
# The template is the first image, so start at 1:
# This dataset_ref list is sorted by visit,
#   which should be equivalent to sorting by MJD
if not SUBTRACTIONS_OF_INJECTED_IMAGE_EXIST:
    for dr in dataset_refs[1:]:
        subtract_and_detect(dr.dataId, template, butler)

## Show cut outs for each subtraction

## Some helper utilities for plotting

In [ ]:
def show_image_on_wcs(calexp, figsize=(8, 8), ax=None, x=None, y=None,
                      pixel_extent=None, stamp_size=None,
                      marker="o", color="red", size=20):
    """
    Show an image with an RA, Dec grid overlaid.  Optionally add markers.
    
    Notes
    -----
    Specifying both pixel_extent and size is undefined.
    """
    if ax is None:
        fig = plt.figure(figsize=figsize)
        plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))
        ax = plt.gca()

    if stamp_size is not None and x is not None and y is not None:    
        half_stamp = stamp_size / 2
        # If x and y are of different types, then user should clarify what they wanted
        if np.isscalar(x):
            first_x = x
            first_y = y
        else:
            first_x = x[0]
            first_y = y[0]
            
        pixel_extent = (int(first_x - half_stamp), int(first_x + half_stamp),
                        int(first_y - half_stamp), int(first_y + half_stamp))
    if pixel_extent is None:
        pixel_extent = (int(calexp.getBBox().beginX), int(calexp.getBBox().endX),
                        int(calexp.getBBox().beginY), int(calexp.getBBox().endY))
    # Image array is y, x.  
    # So we select from the image array in [Y_Begin:Y_End, X_Begin:X_End]
    # But then `extent` is (X_Begin, X_End, Y_Begin, Y_End)

    im = ax.imshow(calexp.image.array[pixel_extent[2]:pixel_extent[3], pixel_extent[0]:pixel_extent[1]],
                   cmap="gray", vmin=-200.0, vmax=400,
                   extent=pixel_extent, origin="lower")
    ax.grid(color="white", ls="solid")
    ax.set_xlabel("Right Ascension")
    ax.set_ylabel("Declination")
    if x is not None and y is not None:
        ax.scatter(x, y, s=size, marker=marker, edgecolor=color, facecolor="none")

### Identify pixel regions to focus on

In [ ]:
def getSiXyFromCalexp(visit_id, calexp, injection_catalog=injection_catalog):
    """
    visit_id: Visit index into catalog that has RA, Dec
    calexp: Determines the WCS frame that converts RA, Dec -> x, y
    """
    this_injection_catalog = injection_catalog[visit_id]
    xy_coords = calexp.getWcs().skyToPixel(geom.SpherePoint(this_injection_catalog[0]["ra"], this_injection_catalog[0]["dec"], geom.degrees))
    return xy_coords    

In [ ]:
template_xy_coords = getSiXyFromCalexp(template.visitInfo.id, template)

In [ ]:
stamp_size = 400

Here's the template

In [ ]:
xy_coords = getSiXyFromCalexp(template.visitInfo.id, template)
show_image_on_wcs(template, x=xy_coords.x, y=xy_coords.y, stamp_size=stamp_size)

Now let's step through the calexps and subtractions

We can see the difference between the location of the star at the template epoch ("green") from the location of the star in the science epoch ("red").

In [ ]:
def plot_calexp_cutout(data_id, template, butler=butler, ax=None, visit_table=visit_table, injection_catalog=injection_catalog, verbose=True):
    calexp = butler.get("calexp", dataId=dr.dataId)
    # Warp the template to get the orientation
    warper_config = WarperConfig()
    warper = Warper.fromConfig(warper_config)

    template_wcs = template.getWcs()
    template_bbox = template.getBBox()
    
    # Add PSF.  I think doing this directly without warping is wrong.  At least the x,y mapping should be updated
    warped_calexp = warper.warpExposure(template_wcs, calexp, destBBox=template_bbox)
    warped_calexp.setPsf(calexp.getPsf())

    template_xy_coords = getSiXyFromCalexp(template.visitInfo.id, warped_calexp, injection_catalog=injection_catalog)
    xy_coords = getSiXyFromCalexp(warped_calexp.visitInfo.id, warped_calexp, injection_catalog=injection_catalog)
    colors = ["green", "red"]
    
    visit_id = calexp.visitInfo.id
    ra, dec = injection_catalog[visit_id][0]["ra"], injection_catalog[visit_id][0]["dec"]
    template_ra, template_dec = injection_catalog[template.visitInfo.id][0]["ra"], injection_catalog[template.visitInfo.id][0]["dec"]
    separation = SkyCoord(ra, dec, unit=u.degree).separation(SkyCoord(template_ra, template_dec, unit=u.degree))

    if verbose:
        print(f"Visit: {visit_id}, MJD: {visit_table.loc[visit_id]['expMidptMJD']:0.6f}, " + \
              f"RA: {ra:0.7f}, Dec: {dec:0.7f}, Displacement: {separation.arcsec:0.6f} arcsec")

    show_image_on_wcs(warped_calexp,
                      ax=ax,
                      x=[template_xy_coords.x, xy_coords.x],
                      y=[template_xy_coords.y, xy_coords.y],
                      color=colors, stamp_size=stamp_size,
                      figsize=(3, 3))
        
    del calexp

In [ ]:
def get_matched_object(dia_src: Table, ra, dec):
    # Match in a simple way
    threshold_dist = 2 / 3600  # arcseconds
    threshold_dist_sq = threshold_dist ** 2

    dist_sq = ((np.rad2deg(dia_src["coord_ra"]) - ra) * np.cos(dia_src["coord_dec"]))**2 + \
              (np.rad2deg(dia_src["coord_dec"]) - dec)**2

    idx, = np.where(dist_sq < threshold_dist_sq)
    try:
        matching_injected = dia_src[idx]
    except:
        matching_injected = None
    
    return matching_injected

In [ ]:
def plot_subtraction_cutout(data_id, template, butler=butler, injection_catalog=injection_catalog, ax=None, verbose=True, figsize=None):
    calexp = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
    dia_src = butler.get("goodSeeingDiff_diaSrc", dataId=data_id)
    # I find Astropy Tables easier to think about than the custom lsst.afw.table SourceCatalog
    dia_src = dia_src.asAstropy()
    # Warp the template to get the orientation
    warper_config = WarperConfig()
    warper = Warper.fromConfig(warper_config)

    template_wcs = template.getWcs()
    template_bbox = template.getBBox()
    
    # Add PSF.  I think doing this directly without warping is wrong.  At least the x,y mapping should be updated
    warped_calexp = warper.warpExposure(template_wcs, calexp, destBBox=template_bbox)
    warped_calexp.setPsf(calexp.getPsf())

    template_xy_coords = getSiXyFromCalexp(template.visitInfo.id, warped_calexp)
    xy_coords = getSiXyFromCalexp(warped_calexp.visitInfo.id, warped_calexp)
    x_array = [template_xy_coords.x, xy_coords.x]
    y_array = [template_xy_coords.y, xy_coords.y]
    colors = ["green", "red"]
    size = [20, 20]
   
    visit_id = calexp.visitInfo.id
    ra, dec = injection_catalog[visit_id][0]["ra"], injection_catalog[visit_id][0]["dec"]
    template_ra, template_dec = injection_catalog[template.visitInfo.id][0]["ra"], injection_catalog[template.visitInfo.id][0]["dec"]
    separation = SkyCoord(ra, dec, unit=u.degree).separation(SkyCoord(template_ra, template_dec, unit=u.degree))
    
    if verbose:
        print(f"Visit: {visit_id}, MJD: {visit_table.loc[visit_id]['expMidptMJD']:0.6f}, " + \
              f"RA: {ra:0.7f}, Dec: {dec:0.7f}, Displacement: {separation.arcsec:0.6f} arcsec")
    
    matching_injected = get_matched_object(dia_src, ra, dec)
    
    columns_of_interest = ["ip_diffim_PsfDipoleFlux_pos_instFlux",
                           "ip_diffim_PsfDipoleFlux_neg_instFlux",
                           "ip_diffim_ClassificationDipole_value"]

    if matching_injected is None or len(matching_injected) < 1:
        print("No matching dia source found for injected object.")
    else:
        print(f"Dipole: ")
        matching_injected[columns_of_interest].pprint(max_width=-1)
        
        dia_src_xy_coords = warped_calexp.getWcs().skyToPixel(geom.SpherePoint(matching_injected["coord_ra"],
                                                                               matching_injected["coord_dec"],
                                                                               geom.radians))
        x_array.append(dia_src_xy_coords.x)
        y_array.append(dia_src_xy_coords.y)
        colors.append("blue")
        size.append(40)

    show_image_on_wcs(warped_calexp, ax=ax, x=x_array, y=y_array, color=colors, size=size, stamp_size=stamp_size, figsize=figsize);
    
    del calexp
    del dia_src

In [ ]:
stamp_size = 100
figsize = (8, 5)
# Note that each image will be shown in its own orientation.
for dr in dataset_refs[1:]:
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1, projection=WCS(template.getWcs().getFitsMetadata()))
    plot_calexp_cutout(dr.dataId, template, butler, ax=plt.gca(), verbose=False);
    plt.subplot(1, 2, 2, projection=WCS(template.getWcs().getFitsMetadata()))
    plot_subtraction_cutout(dr.dataId, template, butler, ax=plt.gca());
    plt.tight_layout()
    plt.show()

Green: Location of star in template  
Red: Location of star in science  
Blue: Location of matching dia_src

The injected star shows up clearly in the subtraction against the original template.

If we subtract the injected template from the injected science we see a dipole

### Appendix.  Snippets of code one might want later

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector